In [ ]:
#  Last amended:06th Sep, 2022
#  Myfolder: /home/ashok/Documents/spark
# Ref:
# Tutorials (slightly dated):
#      https://changhsinlee.com/pyspark-dataframe-basics/
#      https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/
# Cheat Sheet
#      https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf

#  Objectives:
#           Dataframe operations in spark cluster

pyspark APIs<br>
> i)  [DataFrame APIs](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#dataframe-apis)<br>
>> df.select(columnName).where(colObject > 30).orderBy(desc(columnName))<br>
>> df.select(columnName).where("colName > 30").orderBy(desc(columnName))<br>

> ii) [Column APIs](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#column-apis)<br>
>> df.select(df.age.isNull())<br>
>> df.select(df["age"].isNull())<br>
>> df.select(col("age").isNull())<br>

> iii)[Data Tyoes](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#data-types)<br>
> iv) [Functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions)<br>
>> df.select(sum("age"))<br>
>> df.select(sum(col(booleanColumn).cast("int")))<br>
>> <u>but you must import the functions</u>

> v)  [Grouping](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#grouping)<br>

A. Initial operations:
1.0 Start hadoop in a terminal:

            ./allstart.sh
            OR
            ./quick_allstart.sh

## Transfer files to hadoop

In [1]:
# 1.1 Transfer data file 'blackfridayless.csv' to hadoop
#     Linux File folder:  /cdata/misc_datasets/black_friday
#     In Hadoop first make a folder: /user/ashok/datadir 
#     and then transfer the file 'blackfridayless.csv' to 
#     this folder: /user/ashok/datadir

"""

cd ~
hdfs dfs -rm -f -r  /user/ashok/datadir
hdfs dfs -mkdir /user/ashok/datadir
hdfs dfs -put /cdata/misc_datasets/black_friday/blackfridayless.csv  /user/ashok/datadir
hdfs dfs -ls /user/ashok/datadir


"""

!cd ~
!hdfs dfs -rm -f -r  /user/ashok/datadir
!hdfs dfs -mkdir /user/ashok/datadir
!hdfs dfs -put /cdata/misc_datasets/black_friday/blackfridayless.csv  /user/ashok/datadir
!hdfs dfs -ls /user/ashok/datadir




Found 1 items
-rw-r--r--   1 ashok supergroup    4582364 2022-10-11 13:59 /user/ashok/datadir/blackfridayless.csv


## Set jupyter notebook options
Start pyspark with jupyter notebook interface. There is no need to create SparkContext and Spark session. pyspark creates them when starting.

In [2]:
# 1.2 Display multiple outputs from a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# 1.3 Increase cell width to display wide columnar output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_4533/4238029408.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


### Read the csv file from hadoop

In [4]:
###### Read file 'blackfridayless.csv' from hadoop

# 2.0 What is the URL of folder in hadoop where blackfriday file existshaving the file?
#     url: "hdfs://localhost:9000/<folderPath>"


URL_of_folder= "hdfs://localhost:9000/user/ashok/datadir/"

In [5]:
# 2.1 Read the file blackfridayless.csv. Takes time.
#      Use 'spark.read.csv' session object to read file:
#       Here is reading template:

blackfriday = spark.read.csv(
                             path = URL_of_folder + "blackfridayless.csv" ,
                             inferSchema = True,                # True or False
                             header = True,                   # True or False
                             sep = ",",                        # Which one: , ;, | etc
                             ignoreLeadingWhiteSpace = True,    # True or False
                             ignoreTrailingWhiteSpace = True     # True of False
    
                            )

## Explore the dataframe

In [6]:
# 2.2 Show five rows of data:
blackfriday.head(5)
blackfriday.show(3)


[Row(userId=1000001, productId='P00069042', gender='F', age='0-17', occupation=10, cityCategory='A', stayCityYears='2', maritalStatus=0, productCat1=3, productCat2=None, productCat3=None, purchase=8370),
 Row(userId=1000001, productId='P00248942', gender='F', age='0-17', occupation=10, cityCategory='A', stayCityYears='2', maritalStatus=0, productCat1=1, productCat2=6, productCat3=14, purchase=15200),
 Row(userId=1000001, productId='P00087842', gender='F', age='0-17', occupation=10, cityCategory='A', stayCityYears='2', maritalStatus=0, productCat1=12, productCat2=None, productCat3=None, purchase=1422),
 Row(userId=1000001, productId='P00085442', gender='F', age='0-17', occupation=10, cityCategory='A', stayCityYears='2', maritalStatus=0, productCat1=12, productCat2=14, productCat3=None, purchase=1057),
 Row(userId=1000002, productId='P00285442', gender='M', age='55+', occupation=16, cityCategory='C', stayCityYears='4+', maritalStatus=0, productCat1=8, productCat2=None, productCat3=None, 

+-------+---------+------+----+----------+------------+-------------+-------------+-----------+-----------+-----------+--------+
| userId|productId|gender| age|occupation|cityCategory|stayCityYears|maritalStatus|productCat1|productCat2|productCat3|purchase|
+-------+---------+------+----+----------+------------+-------------+-------------+-----------+-----------+-----------+--------+
|1000001|P00069042|     F|0-17|        10|           A|            2|            0|          3|       null|       null|    8370|
|1000001|P00248942|     F|0-17|        10|           A|            2|            0|          1|          6|         14|   15200|
|1000001|P00087842|     F|0-17|        10|           A|            2|            0|         12|       null|       null|    1422|
+-------+---------+------+----+----------+------------+-------------+-------------+-----------+-----------+-----------+--------+
only showing top 3 rows



In [12]:
# 2.3 Show data columns

blackfriday.columns
len(blackfriday.columns)

['userId',
 'productId',
 'gender',
 'age',
 'occupation',
 'cityCategory',
 'stayCityYears',
 'maritalStatus',
 'productCat1',
 'productCat2',
 'productCat3',
 'purchase']

12

In [13]:
# 2.4 Show dtypes:
blackfriday.dtypes


[('userId', 'int'),
 ('productId', 'string'),
 ('gender', 'string'),
 ('age', 'string'),
 ('occupation', 'int'),
 ('cityCategory', 'string'),
 ('stayCityYears', 'string'),
 ('maritalStatus', 'int'),
 ('productCat1', 'int'),
 ('productCat2', 'int'),
 ('productCat3', 'int'),
 ('purchase', 'int')]

In [16]:
# 2.5 Print schema of blackfriday:

blackfriday.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- productId: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- occupation: integer (nullable = true)
 |-- cityCategory: string (nullable = true)
 |-- stayCityYears: string (nullable = true)
 |-- maritalStatus: integer (nullable = true)
 |-- productCat1: integer (nullable = true)
 |-- productCat2: integer (nullable = true)
 |-- productCat3: integer (nullable = true)
 |-- purchase: integer (nullable = true)



In [17]:
blackfriday["age"]

Column<'age'>

In [23]:
blackfriday.select('age','occupation').show(3)

+----+----------+
| age|occupation|
+----+----------+
|0-17|        10|
|0-17|        10|
|0-17|        10|
+----+----------+
only showing top 3 rows



In [26]:
# 3.0 Describe the statistics of data, few columns at a time:

blackfriday.select('age','gender','occupation').describe().show()

+-------+------+------+-----------------+
|summary|   age|gender|       occupation|
+-------+------+------+-----------------+
|  count|100887|100887|           100887|
|   mean|  null|  null|8.086631577903992|
| stddev|  null|  null|6.529747259316683|
|    min|  0-17|     F|                0|
|    max|   55+|     M|               20|
+-------+------+------+-----------------+



In [8]:
# 3.1 Count How many distinct userids are there 
#     Use distinct() and count()

blackfriday.select('userId').distinct().count()

blackfriday.select('age').distinct().show()


5801

+-----+
|  age|
+-----+
|18-25|
|26-35|
| 0-17|
|46-50|
|51-55|
|36-45|
|  55+|
+-----+



In [ ]:
# 3.2 Count how many distinct age-groups exist



In [67]:
# 4.0 How many null values occur in each column

from pyspark.sql.functions import isnan, isnull,col, sum, max, min

In [68]:
# 4.1
for i in blackfriday.columns:
    blackfriday.select(sum(col(i).isNull().alias("nullcol").cast("int")).alias(i)).show()    

+------+
|userId|
+------+
|     0|
+------+

+---------+
|productId|
+---------+
|        0|
+---------+

+------+
|gender|
+------+
|     0|
+------+

+---+
|age|
+---+
|  0|
+---+

+----------+
|occupation|
+----------+
|         0|
+----------+

+------------+
|cityCategory|
+------------+
|           0|
+------------+

+-------------+
|stayCityYears|
+-------------+
|            0|
+-------------+

+-------------+
|maritalStatus|
+-------------+
|            0|
+-------------+

+-----------+
|productCat1|
+-----------+
|          0|
+-----------+

+-----------+
|productCat2|
+-----------+
|      31429|
+-----------+

+-----------+
|productCat3|
+-----------+
|      70100|
+-----------+

+--------+
|purchase|
+--------+
|       0|
+--------+



These columns have null values. Most probably it means that there is no sub-category or sub-categories present. <br>
How would you plan to fill them?<br>
productCat1 :  0 <br>
productCat2 :  31429 <br>
productCat3 :  70100 <br>

In [69]:
# 5.0 Get a list of all integer columns and string columns
#     Use list comprhension along with dtypes:
[    i[0]       for i in blackfriday.dtypes if i[1]=='int']


['userId',
 'occupation',
 'maritalStatus',
 'productCat1',
 'productCat2',
 'productCat3',
 'purchase']

In [70]:
[    i           for i in blackfriday.dtypes if i[1]=='string']

[('productId', 'string'),
 ('gender', 'string'),
 ('age', 'string'),
 ('cityCategory', 'string'),
 ('stayCityYears', 'string')]

In [71]:
# 5.1 Display maximum of productCat2 and productCat3
#     Use select() along with 'max' function


#blackfriday.select('productCat2').max()
blackfriday.select(max('productCat2')).show()
blackfriday.select(max('productCat3')).show()

+----------------+
|max(productCat2)|
+----------------+
|              18|
+----------------+

+----------------+
|max(productCat3)|
+----------------+
|              18|
+----------------+



In [72]:
# 5.2 Find minimum and max values of 'occupation' column

blackfriday.select(max('occupation')).show()
blackfriday.select(min('occupation')).show()

+---------------+
|max(occupation)|
+---------------+
|             20|
+---------------+

+---------------+
|min(occupation)|
+---------------+
|              0|
+---------------+



In [79]:
# 5.3 Fill null values in productCat2 and productCat3 with 999
#     Use df.na.fill()\

blackfriday=blackfriday.na.fill(value=999, subset=["productCat2", "productCat3"])
blackfriday.show()


+-------+---------+------+-----+----------+------------+-------------+-------------+-----------+-----------+-----------+--------+
| userId|productId|gender|  age|occupation|cityCategory|stayCityYears|maritalStatus|productCat1|productCat2|productCat3|purchase|
+-------+---------+------+-----+----------+------------+-------------+-------------+-----------+-----------+-----------+--------+
|1000001|P00069042|     F| 0-17|        10|           A|            2|            0|          3|        999|        999|    8370|
|1000001|P00248942|     F| 0-17|        10|           A|            2|            0|          1|          6|         14|   15200|
|1000001|P00087842|     F| 0-17|        10|           A|            2|            0|         12|        999|        999|    1422|
|1000001|P00085442|     F| 0-17|        10|           A|            2|            0|         12|         14|        999|    1057|
|1000002|P00285442|     M|  55+|        16|           C|           4+|            0|      

In [80]:
# 6.0 Transform spark dataframe to pandas dataframe:
#     Use df.toPandas()

blackfriday.toPandas()


,userId,productId,gender,age,occupation,cityCategory,stayCityYears,maritalStatus,productCat1,productCat2,productCat3,purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,999,999,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6,14,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,999,999,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14,999,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,999,999,7969
...,...,...,...,...,...,...,...,...,...,...,...,...
100882,1003618,P00359742,M,55+,17,A,4+,1,1,5,999,11660
100883,1003618,P00035142,M,55+,17,A,4+,1,1,5,999,11813
100884,1003618,P00207942,M,55+,17,A,4+,1,6,8,999,16020
100885,1003618,P00151142,M,55+,17,A,4+,1,6,16,999,8460


In [87]:
# 6.1 Show a value count of levels of column 'cityCategory':
#      Use groupby and count

blackfriday.groupby('cityCategory').count().show()


+------------+-----+
|cityCategory|count|
+------------+-----+
|           B|42347|
|           C|31071|
|           A|27469|
+------------+-----+



In [88]:
# 7.0 Perform a stratified sampling of data.
#      Stratified sampling be by column: 'cityCategory'
#        Take 80% from 'B' and 20% from 'C'

sample = blackfriday.sampleBy(
                      "cityCategory",      # column that defines strata
                      fractions = {'B' : 0.8, 'C' : 0.2})   # sampling fraction for each stratum

## Using verbs
>select, <br>
><i>select(x).where()</i>,<br>
><i>select().distinct()</i>,<br>
>filter,<br>
>groupby

### select syntax
> DataFrame.select(\*cols)<br>
> cols: column names (string) or expressions (Column). If one of the column names is ‘*’, that column is expanded to include all columns in the current DataFrame.


In [95]:
# 8.0 Show columns 3rd till 5th

blackfriday.select(blackfriday.columns[2:5]).show(3)

+------+----+----------+
|gender| age|occupation|
+------+----+----------+
|     F|0-17|        10|
|     F|0-17|        10|
|     F|0-17|        10|
+------+----+----------+
only showing top 3 rows



### filter syntax
>DataFrame.filter(condition)<br>
>condition: <i>columnObject > 34</i> or string format: <i>"age > 34"</i>
>>  df.age > 3 or col("age") > 3<br>
>>  "age > 3" <br>
>>Logical Operators<br>
>>> If string: AND OR NOT<br>
>>> If columnObject: &, |, ~ <br>

In [102]:
# 8.1 Filter purchases less than 9000
#      Use filter()

blackfriday.filter(blackfriday.purchase < '9000').show()
#condition: purchase < 9000
    

+-------+---------+------+-----+----------+------------+-------------+-------------+-----------+-----------+-----------+--------+
| userId|productId|gender|  age|occupation|cityCategory|stayCityYears|maritalStatus|productCat1|productCat2|productCat3|purchase|
+-------+---------+------+-----+----------+------------+-------------+-------------+-----------+-----------+-----------+--------+
|1000001|P00069042|     F| 0-17|        10|           A|            2|            0|          3|        999|        999|    8370|
|1000001|P00087842|     F| 0-17|        10|           A|            2|            0|         12|        999|        999|    1422|
|1000001|P00085442|     F| 0-17|        10|           A|            2|            0|         12|         14|        999|    1057|
|1000002|P00285442|     M|  55+|        16|           C|           4+|            0|          8|        999|        999|    7969|
|1000005|P00274942|     M|26-35|        20|           A|            1|            1|      

In [103]:
# 8.2 Filter purchases less than 9000 and maritalStatus is 0

blackfriday.filter((blackfriday.purchase < '9000')&(blackfriday.maritalStatus == '0')).show()

+-------+---------+------+-----+----------+------------+-------------+-------------+-----------+-----------+-----------+--------+
| userId|productId|gender|  age|occupation|cityCategory|stayCityYears|maritalStatus|productCat1|productCat2|productCat3|purchase|
+-------+---------+------+-----+----------+------------+-------------+-------------+-----------+-----------+-----------+--------+
|1000001|P00069042|     F| 0-17|        10|           A|            2|            0|          3|        999|        999|    8370|
|1000001|P00087842|     F| 0-17|        10|           A|            2|            0|         12|        999|        999|    1422|
|1000001|P00085442|     F| 0-17|        10|           A|            2|            0|         12|         14|        999|    1057|
|1000002|P00285442|     M|  55+|        16|           C|           4+|            0|          8|        999|        999|    7969|
|1000006|P00231342|     F|51-55|         9|           A|            1|            0|      

In [49]:
# 8.3 Filter purchases less than 9000 or maritalStatus is 0



In [109]:
# 9.0 Combining verbs: select, filter and distinct

blackfriday.select('purchase', 'maritalStatus'). \
            filter((blackfriday.purchase < '9000')). \
            show()

# 10.1
blackfriday.select('purchase', 'maritalStatus'). \
            filter((blackfriday.purchase < '9000')&(blackfriday.maritalStatus == '0')). \
            distinct(). \
            show()


+--------+-------------+
|purchase|maritalStatus|
+--------+-------------+
|    8370|            0|
|    1422|            0|
|    1057|            0|
|    7969|            0|
|    7871|            1|
|    5254|            1|
|    3957|            1|
|    6073|            1|
|    5378|            0|
|    2079|            0|
|    8851|            0|
|    8584|            1|
|    5982|            1|
|    5887|            0|
|    6973|            0|
|    5391|            0|
|    8886|            1|
|    5875|            1|
|    8854|            1|
|    5152|            1|
+--------+-------------+
only showing top 20 rows

+--------+-------------+
|purchase|maritalStatus|
+--------+-------------+
|    8757|            0|
|    4102|            0|
|    3713|            0|
|    5269|            0|
|    6396|            0|
|    7074|            0|
|    8764|            0|
|    6007|            0|
|    7536|            0|
|    7945|            0|
|    4691|            0|
|    5332|            0|

## Aggregation with groupby
Use: <i>.agg({'colName1' : 'mean', 'colName2' : 'sum'})</i> <br>
>With <i>agg()</i> one can use only builtin functions and not any other <i>pyspark.sql.function</i>.<br>
Some common functions are: <i>mean, avg, sum, count, first, last,stddev </i>. There is no need to import builtin function in advance.<br>
For a complete list of builtin functions see [here](https://sparkbyexamples.com/pyspark/pyspark-aggregate-functions/).


In [51]:
# Find max of 'maritalStatus' and max of 'purchase'



In [52]:
# 12. groupby. Can apply sum, min, max, count

airports_df.groupby('tz'). \
           count(). \
           show(3)

# 12.1
airports_df.groupby('tz'). \
            agg({'lat' : 'mean'}). \
            show(3)

NameError: name 'airports_df' is not defined

In [53]:
"""
Unpacking operator in python (*) : 
Ref: https://codeyarns.com/2012/04/26/unpack-operator-in-python/ 

def fox(a,b):  
    return (a *b)  

m = [3,4]  
fox(m)    
fox(*m)   

"""

'\nUnpacking operator in python (*) : \nRef: https://codeyarns.com/2012/04/26/unpack-operator-in-python/ \n\ndef fox(a,b):  \n    return (a *b)  \n\nm = [3,4]  \nfox(m)    \nfox(*m)   \n\n'

In [54]:
# 12.2 One can take the average of columns by passing
#       an unpacked list of column names.

grObject = airports_df.groupby('tz')

avg_cols = ['lat', 'lon']
grObject.avg(*avg_cols).show(3)

NameError: name 'airports_df' is not defined

In [55]:
# 12.3 To call multiple aggregation functions at once, pass a dictionary.
#         The 'key' of dictionary becomes argument to 'value'.
#                             count(*)        avg(lat)      sum(lon)

grObject.agg({'*': 'count', 'lat': 'avg', 'lon':'sum'}).show(2)

NameError: name 'grObject' is not defined

## Column manipulation

In [56]:
# 8. Create new columns in Spark using .withColumn() --mutate
#      New column: altInThousands . 
#      Product of two columns:  'alt' and  'lon' 




In [ ]:
# 9. Save the new file with additional column in parquet form

xyz = airports_df.withColumn('altInThousands', airports_df.alt*airports_df.lon)
xyz.write.parquet("hdfs://localhost:9000/user/ashok/data_files/airports_extra.parquet")

In [ ]:
# 9.1 Delete xyz from spark
import gc
del xyz
gc.collect()    # Delete all cache also

In [ ]:
# 9.2 Read the stored parquet file
df = spark.read.parquet("hdfs://localhost:9000/user/ashok/data_files/airports_extra.parquet")
df.show(3)

## Joining tables

In [ ]:
# 9.3 Read 'weather.csv file into spark from hadoop

URL_of_file= "hdfs://localhost:9000/user/ashok/data_files/nycflights/"
weather_df = spark.read.csv(path = URL_of_file + "weather.csv",
                            inferSchema = True,
                            header = True
                           )
weather_df.show(3)

In [ ]:
# 10. Joins
# Refer: http://www.learnbymarketing.com/1100/pyspark-joins-by-example/
# For example, I can join the two titanic dataframes by the column PassengerId

# 10.1
airports_df.join(weather_df, airports_df.faa==weather_df.origin).show(3)
# 10.2
airports_df.join(weather_df, airports_df.faa==weather_df.origin, how = 'inner').show(3)
# 10.3
airports_df.join(weather_df, airports_df.faa==weather_df.origin, how = 'left').show(3)   # Could also use 'left_outer', 'right', 'full'


## SQL queries against DataFrame

In [ ]:
# 11. Many of the operations can be accessed by writing SQL queries in spark.sql().
# To make an existing Spark dataframe usable for spark.sql(), one needs to
#   register said dataframe as a temporary table.

# 11.1 As an example, we can register the two dataframes as temp tables then
#      join them through spark.sql().

airports_df.createOrReplaceTempView('dfa_temp')
weather_df.createOrReplaceTempView('dfw_temp')

In [ ]:
# 11.2 Simple SQL query. SQLContext is no longer needed. 'spark'
#            session object can be used.

dfj = spark.sql('select * from dfa_temp' )
dfj.show(3)

In [ ]:
# 11.3 Now the SQL join

dfj = spark.sql('select * from dfa_temp a, dfw_temp b where a.faa = b.origin' )
dfj.show(3)


## Misc

In [ ]:
# 12. Drop a columns

airports_df.drop('name').show(3)

# 12.1  Or drop multiple columns

columns_to_drop = ['name', 'lat']
xx =airports_df.drop(*columns_to_drop)
xx.show(3)

In [ ]:
########### I am done ####################